# Python Inventory System - Part 2: Database Configuration

This notebook covers the database configuration for our Inventory Management System, focusing on the `database.py` file. We'll learn how to set up database connections, initialize the database, and manage sessions using SQLModel.

## What We'll Cover

1. Introduction to database engines and sessions
2. Configuring SQLite database connection
3. Initializing the database
4. Working with database sessions
5. Best practices for database management

## 1. Introduction to Database Engines and Sessions

In SQLModel/SQLAlchemy, there are two key concepts:

**Database Engine**:
- The engine is the starting point for any SQLAlchemy application
- It manages the connection pool and the dialect (SQLite, PostgreSQL, etc.)
- Created once per application

**Database Session**:
- Represents a "workspace" for all database operations
- Used to execute queries and track changes to objects
- Typically created per request/operation and closed afterwards

**Why this separation?**
- Engine is expensive to create (connection pooling, dialect setup)
- Sessions are lightweight and should be created frequently
- This architecture is thread-safe and efficient

## 2. Configuring SQLite Database Connection

Let's examine the database configuration code:

In [ ]:
import os
from sqlmodel import SQLModel, create_engine, Session

# Define the database file path
DB_FILE = "inventory.db"
DB_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), DB_FILE)

# Create SQLite URL
SQLITE_URL = f"sqlite:///{DB_PATH}"

# Create engine
engine = create_engine(SQLITE_URL, echo=False, connect_args={"check_same_thread": False})

### Key Components Explained:

1. **Database File Path**:
   ```python
   DB_FILE = "inventory.db"
   DB_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), DB_FILE)
   ```
   - `DB_FILE`: Name of our SQLite database file
   - `DB_PATH`: Full path to the database file, constructed using `os.path` functions
   - `os.path.abspath(__file__)`: Gets absolute path of current file
   - `os.path.dirname()`: Gets the directory containing the file

2. **SQLite URL**:
   ```python
   SQLITE_URL = f"sqlite:///{DB_PATH}"
   ```
   - Format: `sqlite:///<path_to_db_file>`
   - This is the connection string SQLAlchemy uses to connect to SQLite

3. **Engine Creation**:
   ```python
   engine = create_engine(SQLITE_URL, echo=False, connect_args={"check_same_thread": False})
   ```
   - `create_engine()`: Creates the database engine
   - `echo=False`: Disables SQL query logging (set to True for debugging)
   - `connect_args={"check_same_thread": False}`: Allows multiple threads to use the same connection (important for some applications)

## 3. Initializing the Database

The `init_db()` function creates all tables defined in our SQLModel models:

In [ ]:
def init_db():
    """Initialize the database and create tables if they don't exist"""
    SQLModel.metadata.create_all(engine)

# Example usage
print("Creating database tables...")
init_db()
print("Database tables created!")

### How It Works:

- `SQLModel.metadata` contains information about all our models
- `create_all(engine)`:
  - Examines the metadata
  - Generates appropriate CREATE TABLE statements
  - Only creates tables that don't already exist

**Important Notes:**
- This should typically be called once when the application starts
- In production, you might want more control over database migrations (consider Alembic)
- For development, this simple approach works well

## 4. Working with Database Sessions

The `get_session()` function provides a way to work with database sessions:

In [ ]:
def get_session():
    """Get a new database session"""
    with Session(engine) as session:
        yield session

# Example usage
print("Demonstrating session usage:")

# Get a session
session_gen = get_session()
session = next(session_gen)

try:
    # Session is now open and ready for use
    print(f"Session is active: {session.is_active}")
    
    # Normally you would do database operations here
    # For example: products = session.exec(select(Product)).all()
    
finally:
    # The session will be automatically closed when exiting the 'with' block
    # But we need to manually close it here since we used next()
    session.close()
    print(f"Session is active after closing: {session.is_active}")

### Session Management Explained:

1. **Session Creation**:
   ```python
   with Session(engine) as session:
   ```
   - Creates a new session bound to our engine
   - The `with` block ensures the session is properly closed

2. **Generator Function**:
   ```python
   yield session
   ```
   - Makes this function a generator
   - Allows the session to be used in FastAPI dependency injection
   - Ensures the session is properly closed after use

3. **Session Lifecycle**:
   - Sessions should be short-lived
   - Typically used for a single logical operation
   - Must be closed when done (handled automatically by the `with` block)

**Why use a generator?**
- This pattern works particularly well with FastAPI's dependency injection system
- Ensures proper session cleanup even if an error occurs
- Makes testing easier

## 5. Best Practices for Database Management

Here are some important best practices:

1. **Session Lifetime**:
   - Keep sessions short-lived
   - Don't reuse sessions across different operations
   - Always close sessions when done

2. **Thread Safety**:
   - The engine is thread-safe
   - Sessions are not thread-safe - create a new session for each thread

3. **Error Handling**:
   - Always use try/finally or context managers (`with` blocks) with sessions
   - Rollback transactions on errors

4. **Production Considerations**:
   - For production, consider connection pooling
   - Use proper database migration tools (like Alembic)
   - Consider using PostgreSQL instead of SQLite for production

5. **Testing**:
   - Use a separate test database
   - Consider using transactions that can be rolled back after tests

## Summary

In this notebook, we've learned:

1. **Database engines and sessions** - the core components of SQLAlchemy/SQLModel
2. **Configuring SQLite connections** - setting up the database engine
3. **Initializing the database** - creating tables from our models
4. **Session management** - properly handling database sessions
5. **Best practices** - for robust database operations

This configuration forms the backbone of our inventory management system's data layer. In the next notebooks, we'll see how to use these components to perform actual database operations.